In [14]:
!pip install lxml


In [15]:
from bs4 import BeautifulSoup
import requests
import os

CARPETA_DESTINO ='jobs/'
with open("page/Vacantes en Bosch Group.html", "r", encoding="utf-8") as file:
    content = file.read()
    
soup = BeautifulSoup(content, 'lxml')
job_listings = []

job_sections = soup.find_all('section', {'class': 'openings-section opening opening--grouped js-group'})
job_sections


[<section class="openings-section opening opening--grouped js-group" data-qty="1"><header class="opening-header"><ul class="list--dotted title-list"><li><h3 class="opening-title title display--inline-block text--default">Aguascalientes, México</h3></li><li><span class="title">1 empleo</span></li></ul></header><ul class="opening-jobs grid--gutter padding--none js-group-list"><li class="opening-job job column wide-7of16 medium-1of2"><a class="link--block details" href="https://jobs.smartrecruiters.com/BoschGroup/743999906794539-purchasing-quality-automotive-manager?trid=c2ad57d8-2ec6-41b0-aac8-9fb2f04e9fa5"><h4 class="details-title job-title link--block-target">Purchasing Quality Automotive manager</h4><p class="details-desc job-desc"><span class="margin--right--s">Jornada completa</span></p></a></li></ul></section>,
 <section class="openings-section opening opening--grouped js-group" data-qty="1"><header class="opening-header"><ul class="list--dotted title-list"><li><h3 class="opening-t

In [16]:
def limpiar_nombre_archivo(nombre):
    '''Limpiar caracteres no permitidos en nombres de archivos.'''
    caracteres_invalidos = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
    for char in caracteres_invalidos:
        nombre = nombre.replace(char, '-')
    return nombre

In [17]:
# Now you can loop over the sections and extract the details for each job
for section in job_sections:
    location = section.find('h3', {'class': 'opening-title title display--inline-block text--default'}).text
    jobs = section.find_all('li', {'class': 'opening-job job column wide-7of16 medium-1of2'})
    for job in jobs:
        job_link = job.find('a')['href']
        job_title = job.find('h4', {'class': 'details-title job-title link--block-target'}).text


        job_response = requests.get(job_link)
        if job_response.status_code == 200:
                    job_title=limpiar_nombre_archivo(job_title)
                    filename = os.path.join(CARPETA_DESTINO, job_title + ".html")
                    # Guarda el contenido en un archivo .html
                    with open(filename, 'w', encoding='utf-8') as f:
                        f.write(job_response.text)
        else:
                    print(f"No se pudo descargar el trabajo: {job_title}")


In [24]:
from glob import glob
from bs4 import BeautifulSoup
import pandas as pd

path = 'jobs/*.html'
files = glob(path)

job_data_list = []

for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        content = f.read()
    soup = BeautifulSoup(content, 'lxml')

    # Extract job information using BeautifulSoup
    job_data = {}

    # Título del trabajo
    job_title_element = soup.find('h1', class_='job-title')
    job_data['Título del trabajo'] = job_title_element.text.strip() if job_title_element else None

    # Ubicación del trabajo
    job_location_element = soup.find('span', class_='job-detail')
    job_data['Ubicación del trabajo'] = job_location_element.text.strip() if job_location_element else None

    # Tipo de empleo
    employment_type_element = soup.find('li', itemprop='employmentType')
    job_data['Tipo de empleo'] = employment_type_element.text.strip() if employment_type_element else None

    # Entidad legal
    legal_entity_element = soup.find('li', class_='job-detail')
    job_data['Entidad Legal'] = legal_entity_element.text.strip() if legal_entity_element else None

    # Descripción del trabajo
    job_description_element = soup.find('div', itemprop='description')
    job_data['Descripción del trabajo'] = job_description_element.text.strip() if job_description_element else None

    # Calificaciones/Requisitos
    qualifications_element = soup.find('div', itemprop='qualifications')
    qualifications = [qualification.text.strip() for qualification in qualifications_element.find_all('li')] if qualifications_element else None
    job_data['Calificaciones/Requisitos'] = qualifications

    # Información adicional
    additional_information_element = soup.find('div', itemprop='incentives')
    job_data['Información adicional'] = additional_information_element.text.strip() if additional_information_element else None

    if any(job_data.values()):
        job_data_list.append(job_data)

# Convert job data list to DataFrame
tabla = pd.DataFrame(job_data_list)

# Display the concatenated DataFrame
print(tabla)


                                    Título del trabajo Ubicación del trabajo  \
0                     Project Purchasing Administrator                         
1                                   MFE Process Leader                         
2                        Systems Requirements Engineer                         
3    Pre Production Manager Area in Production - BS...                         
4                   Embedded Software Developer Triage                         
..                                                 ...                   ...   
412        Commodity Buyer Senior - Steel Raw Material                         
413                                Logistic Specialist                         
414      Técnico de Soporte a Mantenimiento Maquinados                         
415                       Jr. Purchasing Project Buyer                         
416                         Embedded Software Engineer                         

         Tipo de empleo       Entidad L

In [29]:
import sqlite3
folder='jobs/'
conn = sqlite3.connect('jobs.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS jobs ("Título del trabajo" TEXT, "Ubicación del trabajo" TEXT, "Tipo de empleo" TEXT, "Entidad Legal" TEXT, "Descripción del trabajo" TEXT, "Calificaciones/Requisitos" TEXT, "Información adicional" TEXT)''')
tabla.to_sql('jobs', conn, if_exists='replace', index=False)
conn.commit()
conn.close()





ProgrammingError: Error binding parameter 6: type 'list' is not supported